In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

df = pd.read_excel('data_for_train.xlsx')

df['workload_code'] = df['workload_size'].astype('category').cat.codes

qs_values = [1, 10, 100, 1000, 10000, 100000, 1000000]
qs1_space = qs2_space = qs3_space = qs_values

qs3_allowed = {
    1: qs_values,
    2: qs_values,
    3: qs_values
}

def train_rf_model(target_column):
    X = df[['grouping', 'workload_code', 'qs1', 'qs2', 'qs3']]
    y = df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    r2 = model.score(X_test, y_test)
    print(f"Model R² score for '{target_column}': {r2:.4f}")
    
    return model

model_time = train_rf_model('time')
model_energy = train_rf_model('energy')

groupings = df['grouping'].unique()
workloads = df['workload_size'].unique()
workload_to_code = dict(zip(df['workload_size'], df['workload_code']))

results = []

for grouping in groupings:
    for workload in workloads:
        workload_code = workload_to_code[workload]

        for target in ['time', 'energy']:
            model = model_time if target == 'time' else model_energy
            best_score = None
            best_params = None

            for qs1 in qs1_space:
                for qs2 in qs2_space:
                    for qs3 in qs3_allowed[grouping]:
                        X_candidate = pd.DataFrame({
                            'grouping': [grouping],
                            'workload_code': [workload_code],
                            'qs1': [qs1],
                            'qs2': [qs2],
                            'qs3': [qs3]
                        })
                        score = model.predict(X_candidate)[0]

                        if best_score is None or score < best_score:
                            best_score = score
                            best_params = (qs1, qs2, qs3)

            X_best = pd.DataFrame({
                'grouping': [grouping],
                'workload_code': [workload_code],
                'qs1': [best_params[0]],
                'qs2': [best_params[1]],
                'qs3': [best_params[2]]
            })

            predicted_time = model_time.predict(X_best)[0]
            predicted_energy = model_energy.predict(X_best)[0]

            results.append({
                'grouping': grouping,
                'workload_size': workload,
                'optimized_for': target,
                'qs1': best_params[0],
                'qs2': best_params[1],
                'qs3': best_params[2],
                'predicted_time': predicted_time,
                'predicted_energy': predicted_energy
            })
results_df = pd.DataFrame(results)

summary = []
for workload in workloads:
    workload_results = results_df[results_df['workload_size'] == workload]

    for target, metric in [('time', 'predicted_time'), ('energy', 'predicted_energy')]:
        best_row = workload_results[workload_results['optimized_for'] == target].nsmallest(1, metric).iloc[0]
        summary.append({
            'Best for': target,
            'workload_size': workload,
            'grouping': best_row['grouping'],
            'qs1': best_row['qs1'],
            'qs2': best_row['qs2'],
            'qs3': best_row['qs3'],
            metric: best_row[metric]
        })

summary_df = pd.DataFrame(summary)

with pd.ExcelWriter("optimal_qs_results.xlsx", engine="openpyxl") as writer:
    results_df.to_excel(writer, sheet_name="Optimal Configs", index=False, startrow=0)
    summary_df.to_excel(writer, sheet_name="Optimal Configs", index=False, startrow=len(results_df) + 5)

print("training and opt is done. output in 'optimal_qs_results.xlsx'.\n")
print("results:")
print(results_df)
print("\nSummary of opt configs:")
print(summary_df)


Model R² score for 'time': 0.9729
Model R² score for 'energy': 0.9128
